# Export monthly mean AHF at two sites
- This script is used to export monthly mean AHF from simulations and AH4GUC dataset;
- Simulations: CNTL, TRAF at FR-Capitole and UK-Manchester. 

In [12]:
import rasterio
from rasterio.windows import Window
import pandas as pd
import matplotlib.pyplot as plt
import string
from itertools import product
home_path = '/gws/nopw/j04/duicv/yuansun/'
# ref: https://figshare.com/articles/dataset/Global_1-km_present_and_future_hourly_anthropogenic_heat_flux/12612458/6

In [2]:
lat_list = [43.6045, 53.480821]
lon_list = [1.4454, -2.232823]
site_names = ['FR-Capitole', 'UK-Manchester']
monthly_data = {site: [] for site in site_names}

for month in range(1, 13):
    monthly_file = f'{home_path}dataset/AH4GUC/version6/AHE_2010_{month:02d}_average.tif'
    with rasterio.open(monthly_file) as src:
        for l, site in enumerate(site_names):
            lat_flux, lon_flux = lat_list[l], lon_list[l]
            row, col = src.index(lon_flux, lat_flux)
            window = Window(col, row, 1, 1)
            value = src.read(1, window=window)[0, 0] / 100000  # Adjusting unit per your earlier note
            monthly_data[site].append(value)

# Convert to DataFrame
df_monthly = pd.DataFrame(monthly_data, index=[f'{i:02d}' for i in range(1, 13)]).T
df_monthly.index.name = 'Site'
df_monthly.columns.name = 'Month'
df_monthly


Month,01,02,03,04,05,06,07,08,09,10,11,12
Site,,,,,,,,,,,,
FR-Capitole,48.10851,48.22127,44.45487,42.32866,41.41099,36.99039,36.10105,35.45168,36.42510,38.69040,45.77249,47.45086
UK-Manchester,23.91677,24.29349,24.57351,22.43931,20.98508,19.49859,17.53218,18.27031,19.67391,20.22477,22.84021,22.60166


In [17]:
values_fr = df_monthly.loc['FR-Capitole'].values
values_uk = df_monthly.loc['UK-Manchester'].values

In [29]:
df_fr = pd.read_csv(f'{home_path}0_urban_traffic/output_analysis/capitole/AHF/data_for_figure/FR-Capitole.csv')
df_fr['traffic_contribution'] = 100*df_fr['TRAFFICFLUX'] / df_fr['AHF']
df_fr.head()

,time,month,hour,year,minute,diurnal,URBAN_AC,URBAN_HEAT,VENTILATION,WASTEHEAT,FGR_U,TRAFFICFLUX,AHF,case,traffic_contribution
0,2004-02-20 01:00:00,2,1,2004,0,01:00,0.0,22.241875,6.045159,4.372286,-66.662770,0.0,26.614161,cntl,0.0
1,2004-02-20 01:30:00,2,1,2004,30,01:30,0.0,22.827105,5.971417,4.448375,-66.744280,0.0,27.275478,cntl,0.0
2,2004-02-20 02:00:00,2,2,2004,0,02:00,0.0,23.363012,6.206764,4.565421,-65.932210,0.0,27.928432,cntl,0.0
3,2004-02-20 02:30:00,2,2,2004,30,02:30,0.0,23.899548,6.374155,4.672603,-65.115234,0.0,28.572151,cntl,0.0
4,2004-02-20 03:00:00,2,3,2004,0,03:00,0.0,24.393486,6.538073,4.779910,-65.081230,0.0,29.173395,cntl,0.0


In [30]:
df_uk = pd.read_csv(f'{home_path}0_urban_traffic/output_analysis/manchester/AHF/data_for_figure/UK-Manchester.csv')
df_uk['traffic_contribution'] = 100*df_uk['TRAFFICFLUX_traffic'] / df_uk['AHF_traffic']
df_uk['time'] = pd.to_datetime(df_uk['time']) - pd.DateOffset(hours=1)
df_uk['month'] = df_uk['time'].dt.month
df_uk.head()

,time,FSH_U_cntl,URBAN_AC_cntl,URBAN_HEAT_cntl,VENTILATION_cntl,WASTEHEAT_cntl,FGR_U_cntl,FSH_U_traffic,URBAN_AC_traffic,URBAN_HEAT_traffic,...,FGR_U_traffic,TRAFFICFLUX_traffic,TRAFFICFLUX_cntl,AHF_cntl,AHF_traffic,hour,minute,diurnal,traffic_contribution,month
0,2022-01-01 00:00:00,-26.170258,0.0,6.337677,1.505959,1.300586,4.305870,-24.011946,0.0,6.337677,...,7.285527,8.433395,0,7.638264,16.071660,1,0,01:00,52.473702,1
1,2022-01-01 01:00:00,-32.742588,0.0,6.229676,1.245074,1.267536,2.255315,-31.170126,0.0,6.210087,...,5.714578,8.427978,0,7.497211,15.905600,2,0,02:00,52.987489,1
2,2022-01-01 02:00:00,-43.709030,0.0,6.177687,1.094389,1.245935,-6.408768,-42.758736,0.0,6.155823,...,-5.389267,4.213982,0,7.423622,11.611822,3,0,03:00,36.290443,1
3,2022-01-01 03:00:00,-43.593456,0.0,6.313346,1.046665,1.235537,-14.708106,-42.123547,0.0,6.298296,...,-14.268779,4.213904,0,7.548883,11.743364,4,0,04:00,35.883282,1
4,2022-01-01 04:00:00,-42.983097,0.0,6.431928,1.324741,1.262669,-16.880140,-41.499035,0.0,6.416971,...,-16.301218,4.213904,0,7.694597,11.890535,5,0,05:00,35.439149,1


In [39]:
months = range(1, 13)
sites = ['FR-Capitole', 'UK-Manchester']
cases = ['AH4GUC', 'cntl', 'traffic']
variables = ['AHF', 'traffic_contribution']
case_list = ['cntl', 'traffic']
# Create all combinations
combinations = list(product(months, cases, sites, variables))

# Create DataFrame
df_combined = pd.DataFrame(combinations, columns=['month', 'case', 'site',  'variable'])
df_combined['value'] = None
df_combined.loc[
    (df_combined['case'] == 'AH4GUC') & (df_combined['site'] == 'FR-Capitole') & (df_combined['variable'] == 'AHF'),
    'value'
] = values_fr
df_combined.loc[
    (df_combined['case'] == 'AH4GUC') & (df_combined['site'] == 'UK-Manchester') & (df_combined['variable'] == 'AHF'),
    'value'
] = values_uk
for case in case_list:
    df_fr_case = df_fr[df_fr['case'] == case]
    for variable in variables:
        df_combined.loc[(df_combined['case'] == case) & (df_combined['site'] == 'FR-Capitole') & (df_combined['variable'] == variable),'value'] = df_fr_case.groupby('month')[variable].mean().values
for case in case_list:
    df_uk_case = df_uk[['month', f'AHF_{case}']].groupby('month').mean()
    df_combined.loc[(df_combined['case'] == case) & (df_combined['site'] == 'UK-Manchester') & (df_combined['variable'] == 'AHF'),'value'] = df_uk_case.values
df_combined.loc[(df_combined['case'] == 'traffic') & (df_combined['site'] == 'UK-Manchester') & (df_combined['variable'] == 'traffic_contribution'),'value'] = df_uk[['month', 'traffic_contribution']].groupby('month').mean().reset_index()['traffic_contribution'].values

df_combined.to_csv('data_for_figure/monthly_mean.csv', index=False)
df_combined

,month,case,site,variable,value
0,1,AH4GUC,FR-Capitole,AHF,48.10851
1,1,AH4GUC,FR-Capitole,traffic_contribution,None
2,1,AH4GUC,UK-Manchester,AHF,23.91677
3,1,AH4GUC,UK-Manchester,traffic_contribution,None
4,1,cntl,FR-Capitole,AHF,19.617936
...,...,...,...,...,...
139,12,cntl,UK-Manchester,traffic_contribution,None
140,12,traffic,FR-Capitole,AHF,35.298384
141,12,traffic,FR-Capitole,traffic_contribution,62.766758
142,12,traffic,UK-Manchester,AHF,41.10583
